<a href="https://colab.research.google.com/github/DavidAntonio7/web_scraping_Bola_de_ouro-/blob/main/df_Bola_de_Ouro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

In [2]:
def recuperar_html(site,ano):
  url = site
  try:
      response = requests.get(url)
      # Verifique se a solicitação foi bem-sucedida (código de status 200)
      if response.status_code == 200:
          html = response.content.decode('utf-8')
          return html
      else:
          print("Erro no código de status HTTP:", response.status_code)
          print('/n' + str(ano) + '/n')
  except requests.exceptions.RequestException as e:
      print("Erro:", e)
      print('/n' + str(ano) + '/n')
  return False


In [3]:
nomeados = []
for ano in range(1956,2023+1):
  site = 'https://www-topendsports-com.translate.goog/sport/soccer/awards/ballondor-' + str(ano) + '.htm?_x_tr_sl=en&_x_tr_tl=pt-PT&_x_tr_hl=pt-PT&_x_tr_pto=tc'
  html = recuperar_html(site,ano)
  if html != False:
    bs = BeautifulSoup(html,'html.parser')
    tags = bs.find_all('td')
    if ano != 2022:
      for i in range(int(len(tags)/5)):
        nomeados.append([ano,tags[i*5].get_text(),(tags[(i*5)+1].get_text()),(tags[(i*5)+2].get_text()),(tags[(i*5)+3].get_text()),tags[(i*5)+4].get_text()])
    else:
      for i in range(int(len(tags)/6)):
        nomeados.append([ano,tags[i*6].get_text(),(tags[(i*6)+1].get_text()),(tags[(i*6)+3].get_text()),(tags[(i*6)+4].get_text()),tags[(i*6)+5].get_text()])

In [4]:
df_nomeados = pd.DataFrame(nomeados,columns=['Year','Order','Player','Nationality','Club','Points'])

In [5]:
#dados com problemas no site
df_nomeados.loc[55, 'Points'] = '6'
df_nomeados.loc[149, 'Player'] = 'Denis Law'
df_nomeados.loc[149, 'Club'] = 'Torino'
df_nomeados.loc[149, 'Nationality'] = 'Scotland'
df_nomeados.loc[262, 'Points'] = '3'
df_nomeados.loc[1909, 'Points'] = '2'
df_nomeados.drop(1005, inplace = True)

In [6]:
posicoes = []
for ano in df_nomeados['Year'].unique():
  pontuacao = 0
  posicao = 0
  numero_de_nomeados = 0
  for indice,nomeado in df_nomeados[df_nomeados['Year'] == ano].iterrows():
    if numero_de_nomeados == 0:
      posicao = 1
    elif (pontuacao != nomeado.Points):
      posicao = numero_de_nomeados + 1
    pontuacao = nomeado.Points
    numero_de_nomeados +=1
    posicoes.append(posicao)

In [7]:
df_nomeados.Order = posicoes

In [8]:
df_nomeados.Year = df_nomeados.Year.astype(int)

In [9]:
df_nomeados.to_csv('nominees.csv',index=False)